- To rerun any S-LOCAL network you need a minimum of 32GB of RAM/VRAM memory on the device used for training
- Average time to run a 100 epoch on a Nvidia Tesla V100 GPU is 20 minutes
- A faster implementation of the Locally Connected Layer is available in the Keras API : 
- https://keras.io/api/layers/locally_connected_layers/locall_connected2d/

In [1]:
INIT_PATH = "."


# To use google colab processing uncomment following lines

#INIT_PATH = "MyDrive/inference_and_learning"
#from google.colab import drive
#MOUNT_PATH = "/content/drive"
#drive.mount(MOUNT_PATH)
#INIT_PATH = MOUNT_PATH + "/" + INIT_PATH

In [2]:
import sys
sys.path.append(INIT_PATH)
import os 
from os.path import exists
from typing import Tuple, List

In [5]:
import importlib
from networks import NetworkTrainer,NetworkOptimizer
import pandas as pd
import torch
torch.set_default_dtype(torch.float16)

0

In [4]:
DATA_PATH = "."
if (False):
    import torchvision
    import torch
    torch.set_default_dtype(torch.float16)
    torchvision.datasets.CIFAR10(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.CIFAR100(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True)

    torchvision.datasets.CIFAR10(root=DATA_PATH +'/data', download=True,train = False)
    torchvision.datasets.CIFAR100(root=DATA_PATH +'/data', download=True ,train = False) 
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True, split = "test")


In [5]:
table2 = pd.read_csv(INIT_PATH+"/csv/table2.csv")
network = pd.read_json(INIT_PATH+"/network_description/table2.json")
for dataset in ["CIFAR-10","SVHN","CIFAR-100" ] : #
  for line in range(table2.__len__()):
        if(table2.loc[line,dataset] == "NaN"):
            continue
        netline = table2.iloc[line]
        print(f"Starting {netline.Model} with {netline.Training_Method} on {dataset}")
        ID= "_".join([netline.Model,netline.Training_Method,dataset])
        network_data = network.loc[ID == network["name"]].iloc[0].to_dict()
        
        #if ID[0] == "S-LOCAL":
            #continue
        best,changed = NetworkOptimizer(network_data,True,True,DATA_PATH)
        print(f"Found best Accuracy : {best['accuracy']} % for {'_'.join(ID)}\n")
        if changed :    
            os.remove(INIT_PATH + f"/network_saved/table_2_{'_'.join(ID)}.pth")
            network.loc[
                (network["model"]== netline.Model) &
                (network["optimizer"]==netline.Training_Method) &
                (network["dataset"]==dataset)].iloc[0] = best
            network.to_json(INIT_PATH+"/network_description/table2.json")
        

Starting S-FC with B-lasso(B=0) on CIFAR-10
Calculating base accuracy...
Training Network: S-FC, CIFAR-10, B-lasso using : 230.60269 M parameters on cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 210.00 MiB (GPU 0; 7.78 GiB total capacity; 1.79 GiB already allocated; 5.25 MiB free; 1.80 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
table2 = pd.read_csv(INIT_PATH+"/csv/table2.csv")
network = pd.read_json(INIT_PATH+"/network_description/table2.json")
optimize = True
for dataset in ["CIFAR-10","SVHN","CIFAR-100" ] : #
  for line in range(table2.__len__()):
        if(table2.loc[line,dataset] == "NaN"):
            continue
        netline = table2.iloc[line]
        network_data = network.loc[
            (network["model"]== netline.Model) &
            (network["optimizer"]==netline.Training_Method) &
            (network["dataset"]==dataset)].iloc[0].to_dict()
        ID = [network_data[i] for i in ["model","optimizer","dataset"]]
        # optimize
        if optimize : 
          if ID[0] == "S-LOCAL":
            continue
          best,changed = NetworkOptimizer(network_data,True,True,DATA_PATH)
          print(f"Found best Accuracy : {best['accuracy']} % for {'_'.join(ID)}\n")
          if changed : 
            os.remove(INIT_PATH + f"/network_saved/table_2_{'_'.join(ID)}.pth")
            network.loc[
              (network["model"]== netline.Model) &
              (network["optimizer"]==netline.Training_Method) &
              (network["dataset"]==dataset)].iloc[0] = best
        
        if not os.path.exists(INIT_PATH + f"/network_saved/table_2_{'_'.join(ID)}.pth"):
          if ID[0] == "S-LOCAL":
            continue
          PATH = INIT_PATH + f"/network_saved/table_2_{'_'.join(ID)}.pth"
          acc,_ = NetworkTrainer(network_data,True,False,DATA_PATH,PATH)
          table2.loc[line,dataset] = str(acc) + "%"
          table2.to_csv("csv/table2.csv",index=False)
        